### Load libs

In [1]:
import pandas as pd
import json
import numpy as np
import random
from pandarallel import pandarallel
import re

pandarallel.initialize()


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
n = 500
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Load data

In [5]:
term = 'AmeriCredit Automobile Receivables Trust 2017-1 Data Tape'
finder = re.compile('\d{4,}\W\d{1,}')
add_id = re.findall(finder, term)[0]
add_id

'2017-1'

In [6]:
# load abs
folder = 'data/combined/'
file = '{}.csv'.format(term)
path = folder + file
data = pd.read_csv(path)
data.shape


/anaconda3/envs/kn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1509167, 73)

In [39]:
# load mapper
m_folder = 'data/dictionary/mapper/'
m_file = 'mapper.json'
m_path = m_folder + m_file
with open(m_path) as f:
    mapper = json.load(f)
m_cols = list(mapper.keys())
    

### Formatting

In [8]:
def convert_id(row, column):
    
    """
    Convert ids
    """
    
    init = str(row[column])
    
    cleaned = init.replace('=', '').replace('"', '').strip()
    final = cleaned + '-' + add_id
    
    return final
    

In [9]:
def reorder_date(row, column):
    
    """
    Reorder date
    """
    
    init = str(row[column])
    if init != '-':
        if '/' not in init:
            y = init[6:10]
            m = init[0:2]
            d = init[3:5]
            date = y + '-' + m + '-' + d
        elif '/' in init:
            y = init[3:7]
            m = init[0:2]
            date = y + '-' + m
    else:
        date = ''
    
    return date
        

In [10]:
data['ID'] = data.parallel_apply(convert_id, args = ('assetNumber', ), axis = 1)


In [11]:
# date cols
date_cols = ['reportingPeriodBeginningDate', 'reportingPeriodEndingDate', 'originationDate', 'loanMaturityDate', 'originalFirstPaymentDate', 'interestPaidThroughDate', 'zeroBalanceEffectiveDate', 'mostRecentServicingTransferReceivedDate', 'DemandResolutionDate'] 


In [12]:
for col in date_cols:
    print(col)
    data['{}R'.format(col)] = data.parallel_apply(reorder_date, args = (col, ), axis = 1)
    

reportingPeriodBeginningDate
reportingPeriodEndingDate
originationDate
loanMaturityDate
originalFirstPaymentDate
interestPaidThroughDate
zeroBalanceEffectiveDate
mostRecentServicingTransferReceivedDate
DemandResolutionDate


In [13]:
replacer_cols = ['originalLoanAmount', 'originalLoanTerm', 'originalInterestRatePercentage', 'gracePeriodNumber', 'obligorCreditScore', 'paymentToIncomePercentage', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'repossessedProceedsAmount']    


In [14]:
data[replacer_cols] = data[replacer_cols].replace('-', np.nan)

In [15]:
# clean cols
clean_cols = ['currentDelinquencyStatus']
for col in clean_cols:
    data[col] = data[col].str.strip()
    data[col] = data[col].astype(float)
    

In [16]:
def replace_val(row, column):
    
    """
    Replace numeric values
    """
    
    init = str(row[column]).strip().replace(';', '')
    if init in ['0', '1', '2', '3', '4', '5', '98', '99']:
        mapped = mapper[column][init]
        return mapped
    else:
        if init[0] in ['0', '1', '2', '3', '4', '5']:
            use = init[0]
        elif init == '-':
            use_keys = list(mapper[column].keys())
            if '98' in use_keys:
                use = '98'
            elif '99' in use_keys:
                use = '99'
        else:
            use = init
        mapped = mapper[column][use]
    return mapped
    

In [17]:
for col in m_cols:
    print(col)
    new_col = col + 'M'
    data[new_col] = data.parallel_apply(replace_val, args = (col, ), axis = 1)
    

interestCalculationTypeCode
originalInterestRateTypeCode
paymentTypeCode
subvented
vehicleNewUsedCode
vehicleTypeCode
vehicleValueSourceCode
obligorIncomeVerificationLevelCode
obligorEmploymentVerificationCode
servicingAdvanceMethodCode
zeroBalanceCode
assetSubjectDemandStatusCode
repurchaseReplacementReasonCode
modificationTypeCode


In [18]:
def acct_status(row, b_col, e_col, zero_col, thresh):
    
    """
    Create karus account status
    """
    
    b = float(row[b_col])
    e = float(row[e_col])
    z = str(row[zero_col])
    
    if z in ['Charged-off', 'Repurchased or Replaced']:
        res = z
        return res
    if b < thresh and e < thresh:
        res = 'Prepaid or Matured'
        return res
    if z in ['Unavailable', 'Prepaid or Matured']:
        res = z
        return res
    

In [19]:
b_col = 'reportingPeriodBeginningLoanBalanceAmount'
e_col = 'nextReportingPeriodPaymentAmountDue'
z_col = 'zeroBalanceCodeM'
thresh = 50


In [20]:
data['accountStatus'] = data.parallel_apply(acct_status, args = (b_col, e_col, z_col, thresh, ), axis = 1)

In [21]:
data['accountStatus'].value_counts()

Unavailable                1456237
Charged-off                26733  
Prepaid or Matured         26159  
Repurchased or Replaced    38     
Name: accountStatus, dtype: int64

### Application

In [22]:
all_ids = list(data['ID'].unique())
print_vals = list(range(0, len(all_ids), 100))


In [36]:
loc_cols = list(set(['originatorName', 'primaryLoanServicerName', 'originationDateR', 'originalLoanAmount', 'originalLoanTerm', 'loanMaturityDateR', 'originalInterestRatePercentage', 'interestCalculationTypeCodeM', 'originalInterestRateTypeCodeM', 'originalInterestOnlyTermNumber', 'originalFirstPaymentDateR', 'underwritingIndicator', 'paymentTypeCodeM', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleNewUsedCodeM', 'vehicleModelYear', 'vehicleTypeCodeM', 'vehicleValueAmount', 'vehicleValueSourceCodeM', 'obligorCreditScoreType', 'obligorIncomeVerificationLevelCodeM', 'obligorEmploymentVerificationCodeM', 'coObligorIndicator', 'paymentToIncomePercentage', 'obligorGeographicLocation', 'assetAddedIndicator', 'reportingPeriodModificationIndicator', 'servicingAdvanceMethodCodeM', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'scheduledInterestAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'assetSubjectDemandIndicator', 'zeroBalanceEffectiveDateR']))         
min_max_cols = list(set(['reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR', 'remainingTermToMaturityNumber', 'obligorCreditScore', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue']))
max_cols = list(set(['gracePeriodNumber', 'interestPaidThroughDateR', 'mostRecentServicingTransferReceivedDateR', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'currentDelinquencyStatus']))
all_vals = list(set(['subventedM', 'assetSubjectDemandIndicator', 'assetSubjectDemandStatusCodeM', 'repurchaserName', 'repurchaseReplacementReasonCodeM', 'modificationTypeCodeM', 'repossessedIndicator', 'zeroBalanceCodeM', 'accountStatus']))
sum_cols = list(set(['servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'repossessedProceedsAmount', 'gracePeriodNumber']))
event_cols = list(set(['accountStatus', 'zeroBalanceCodeM', 'chargedoffPrincipalAmount', 'reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR']))
all_numeric = list(set(min_max_cols + max_cols + sum_cols))


In [24]:
# force convert cols to numeric
for col in all_numeric:
    print(col)
    data[col] = pd.to_numeric(data[col], errors='coerce')

remainingTermToMaturityNumber
otherServicerFeeRetainedByServicer
mostRecentServicingTransferReceivedDateR
actualPrincipalCollectedAmount
servicingFeePercentage
chargedoffPrincipalAmount
paymentExtendedNumber
otherAssessedUncollectedServicerFeeAmount
obligorCreditScore
nextReportingPeriodPaymentAmountDue
servicingFlatFeeAmount
reportingPeriodEndingDateR
actualOtherCollectedAmount
totalActualAmountPaid
interestPaidThroughDateR
actualInterestCollectedAmount
reportingPeriodBeginningLoanBalanceAmount
servicerAdvancedAmount
gracePeriodNumber
repurchaseAmount
repossessedProceedsAmount
recoveredAmount
reportingPeriodBeginningDateR
currentDelinquencyStatus


In [23]:
#_id = '0001694010 - 000010'
#all_ids = ['0001694010 - 000010']
id_col = 'ID'
status_col = 'accountStatus'
values = ['Charged-off', 'Prepaid or Matured', 'Repurchased or Replaced']


In [24]:
holder = []
counter = 0

for _id in all_ids:
    
    try:
        #print(_id)
        counter = counter + 1
        if counter in print_vals:
            print(counter, counter/len(all_ids))
            print('------------------------------')

        df = data[data[id_col] == _id].reset_index(drop = True)
        df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)

        # dict
        account_dict = {}
        account_dict['id'] = _id
        account_dict['records'] = len(df)

        # current status of loan
        for col in loc_cols:
            account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
        for col in min_max_cols:
            _min = df[col].min()
            _max = df[col].max()
            account_dict['{}MinCurrent'.format(col)] = _min
            account_dict['{}MaxCurrent'.format(col)] = _max
        for col in max_cols:
            account_dict['{}MaxCurrent'.format(col)] = df[col].max()
        for col in all_vals:
            vals = list(df[col].unique())
            use_vals = ' | '.join(str(val) for val in vals)
            account_dict['{}ValsCurrent'.format(col)] = use_vals
        for col in sum_cols:
            account_dict['{}SumCurrent'.format(col)] = df[col].sum()

        # event information
        init_vals = list(df[status_col].unique())
        inter = list(set(values).intersection(init_vals))
        if len(inter) > 0:
            account_dict['eventOccurred'] = 1
            n = df[status_col].where(df[status_col].isin(values)).last_valid_index()
            n_bool = True
            single = df.loc[[n]]
            for col in event_cols:
                account_dict['{}Event'.format(col)] = single[col].iloc[0]

            # prior to event
            init = n+1
            sub = df[init:len(df)]
            sub.reset_index(drop = True, inplace = True)
            account_dict['priorHistory'] = len(sub)
            sub_bool = True
            if len(sub) > 0:
                for col in loc_cols:
                    account_dict['{}LocPrior'.format(col)] = sub[col].iloc[0]
                for col in min_max_cols:
                    _min = sub[col].min()
                    _max = sub[col].max()
                    account_dict['{}MinPrior'.format(col)] = _min
                    account_dict['{}MaxPrior'.format(col)] = _max
                for col in max_cols:
                    account_dict['{}MaxPrior'.format(col)] = sub[col].max()
                for col in all_vals:
                    vals = list(sub[col].unique())
                    use_vals = ' | '.join(str(val) for val in vals)
                    account_dict['{}ValsPrior'.format(col)] = use_vals
                for col in sum_cols:
                    account_dict['{}SumPrior'.format(col)] = sub[col].sum()

                # random
                len_sub = len(sub)
                s = random.randint(0, len_sub)
                if s == len_sub:
                    s = s -1
                r_sub = sub[s:len_sub].reset_index(drop = True)
                account_dict['randomIndex'] = s
                for col in loc_cols:
                    account_dict['{}LocRandom'.format(col)] = r_sub[col].iloc[0]
                for col in min_max_cols:
                    _min = r_sub[col].min()
                    _max = r_sub[col].max()
                    account_dict['{}MinRandom'.format(col)] = _min
                    account_dict['{}MaxRandom'.format(col)] = _max
                for col in max_cols:
                    account_dict['{}MaxRandom'.format(col)] = r_sub[col].max()
                for col in all_vals:
                    vals = list(r_sub[col].unique())
                    use_vals = ' | '.join(str(val) for val in vals)
                    account_dict['{}ValsRandom'.format(col)] = use_vals
                for col in sum_cols:
                    account_dict['{}SumRandom'.format(col)] = r_sub[col].sum()

            # if event is first row of sub       
            else:
                for col in loc_cols:
                    account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
                for col in min_max_cols:
                    _min = df[col].min()
                    _max = df[col].max()
                    account_dict['{}MinPrior'.format(col)] = _min
                    account_dict['{}MaxPrior'.format(col)] = _max
                for col in max_cols:
                    account_dict['{}MaxPrior'.format(col)] = df[col].max()
                for col in all_vals:
                    vals = list(df[col].unique())
                    use_vals = ' | '.join(str(val) for val in vals)
                    account_dict['{}ValsPrior'.format(col)] = use_vals
                for col in sum_cols:
                    account_dict['{}SumPrior'.format(col)] = df[col].sum()

        # if no event        
        else:
            account_dict['eventOccurred'] = 0
            account_dict['priorHistory'] = len(df)
            sub_bool = False
            n_bool = False
            for col in event_cols:
                account_dict['{}Event'.format(col)] = np.nan
            for col in loc_cols:
                account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
            for col in min_max_cols:
                _min = df[col].min()
                _max = df[col].max()
                account_dict['{}MinPrior'.format(col)] = _min
                account_dict['{}MaxPrior'.format(col)] = _max
            for col in max_cols:
                account_dict['{}MaxPrior'.format(col)] = df[col].max()
            for col in all_vals:
                vals = list(df[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsPrior'.format(col)] = use_vals
            for col in sum_cols:
                account_dict['{}SumPrior'.format(col)] = df[col].sum()

            # random set to nan
            account_dict['randomIndex'] = np.nan
            for col in loc_cols:
                account_dict['{}LocRandom'.format(col)] = np.nan
            for col in min_max_cols:
                account_dict['{}MinRandom'.format(col)] = np.nan
                account_dict['{}MaxRandom'.format(col)] = np.nan
            for col in max_cols:
                account_dict['{}MaxRandom'.format(col)] = np.nan
            for col in all_vals:
                account_dict['{}ValsRandom'.format(col)] = np.nan
            for col in sum_cols:
                account_dict['{}SumRandom'.format(col)] = np.nan

        holder.append(account_dict)
    except:
        print('ERROR {}'.format(_id))


100 0.0012157167866173896
------------------------------
200 0.0024314335732347793
------------------------------
300 0.003647150359852169
------------------------------
400 0.004862867146469559
------------------------------
500 0.006078583933086948
------------------------------
600 0.007294300719704338
------------------------------
700 0.008510017506321728
------------------------------
800 0.009725734292939117
------------------------------
900 0.010941451079556507
------------------------------
1000 0.012157167866173896
------------------------------
1100 0.013372884652791286
------------------------------
1200 0.014588601439408676
------------------------------
1300 0.015804318226026064
------------------------------
1400 0.017020035012643455
------------------------------
1500 0.018235751799260843
------------------------------
1600 0.019451468585878234
------------------------------
1700 0.020667185372495622
------------------------------
1800 0.021882902159113014
------------

14700 0.17871036763275627
------------------------------
14800 0.17992608441937366
------------------------------
14900 0.18114180120599105
------------------------------
15000 0.18235751799260844
------------------------------
15100 0.18357323477922582
------------------------------
15200 0.1847889515658432
------------------------------
15300 0.1860046683524606
------------------------------
15400 0.187220385139078
------------------------------
15500 0.18843610192569538
------------------------------
15600 0.1896518187123128
------------------------------
15700 0.19086753549893018
------------------------------
15800 0.19208325228554757
------------------------------
15900 0.19329896907216496
------------------------------
16000 0.19451468585878234
------------------------------
16100 0.19573040264539973
------------------------------
16200 0.19694611943201712
------------------------------
16300 0.1981618362186345
------------------------------
16400 0.1993775530052519
------------

29200 0.35498930169227777
------------------------------
29300 0.35620501847889513
------------------------------
29400 0.35742073526551255
------------------------------
29500 0.35863645205212996
------------------------------
29600 0.3598521688387473
------------------------------
29700 0.36106788562536474
------------------------------
29800 0.3622836024119821
------------------------------
29900 0.3634993191985995
------------------------------
30000 0.3647150359852169
------------------------------
30100 0.3659307527718343
------------------------------
30200 0.36714646955845165
------------------------------
30300 0.36836218634506906
------------------------------
30400 0.3695779031316864
------------------------------
30500 0.37079361991830384
------------------------------
30600 0.3720093367049212
------------------------------
30700 0.3732250534915386
------------------------------
30800 0.374440770278156
------------------------------
30900 0.3756564870647734
----------------

43800 0.5324839525384166
------------------------------
43900 0.533699669325034
------------------------------
44000 0.5349153861116515
------------------------------
44100 0.5361311028982688
------------------------------
44200 0.5373468196848862
------------------------------
44300 0.5385625364715037
------------------------------
44400 0.539778253258121
------------------------------
44500 0.5409939700447384
------------------------------
44600 0.5422096868313557
------------------------------
44700 0.5434254036179732
------------------------------
44800 0.5446411204045906
------------------------------
44900 0.5458568371912079
------------------------------
45000 0.5470725539778253
------------------------------
45100 0.5482882707644428
------------------------------
45200 0.5495039875510601
------------------------------
45300 0.5507197043376775
------------------------------
45400 0.5519354211242948
------------------------------
45500 0.5531511379109123
-------------------------

58500 0.7111943201711729
------------------------------
58600 0.7124100369577903
------------------------------
58700 0.7136257537444077
------------------------------
58800 0.7148414705310251
------------------------------
58900 0.7160571873176425
------------------------------
59000 0.7172729041042599
------------------------------
59100 0.7184886208908773
------------------------------
59200 0.7197043376774946
------------------------------
59300 0.720920054464112
------------------------------
59400 0.7221357712507295
------------------------------
59500 0.7233514880373468
------------------------------
59600 0.7245672048239642
------------------------------
59700 0.7257829216105816
------------------------------
59800 0.726998638397199
------------------------------
59900 0.7282143551838164
------------------------------
60000 0.7294300719704337
------------------------------
60100 0.7306457887570511
------------------------------
60200 0.7318615055436686
-------------------------

73200 0.8899046878039292
------------------------------
73300 0.8911204045905465
------------------------------
73400 0.892336121377164
------------------------------
73500 0.8935518381637814
------------------------------
73600 0.8947675549503987
------------------------------
73700 0.8959832717370162
------------------------------
73800 0.8971989885236336
------------------------------
73900 0.8984147053102509
------------------------------
74000 0.8996304220968683
------------------------------
74100 0.9008461388834857
------------------------------
74200 0.9020618556701031
------------------------------
74300 0.9032775724567205
------------------------------
74400 0.9044932892433378
------------------------------
74500 0.9057090060299553
------------------------------
74600 0.9069247228165727
------------------------------
74700 0.90814043960319
------------------------------
74800 0.9093561563898074
------------------------------
74900 0.9105718731764249
--------------------------

In [25]:
master = pd.DataFrame(holder)

In [26]:
master['accountStatusEvent'].value_counts(dropna = False)

Prepaid or Matured    49833
NaN                   28105
Charged-off           4318 
Name: accountStatusEvent, dtype: int64

In [27]:
master.head()

,id,records,vehicleManufacturerNameLocCurrent,primaryLoanServicerNameLocCurrent,originalInterestRateTypeCodeMLocCurrent,vehicleNewUsedCodeMLocCurrent,originalLoanAmountLocCurrent,obligorIncomeVerificationLevelCodeMLocCurrent,underwritingIndicatorLocCurrent,otherPrincipalAdjustmentAmountLocCurrent,originationDateRLocCurrent,obligorCreditScoreTypeLocCurrent,obligorGeographicLocationLocCurrent,paymentToIncomePercentageLocCurrent,nextReportingPeriodPaymentAmountDueLocCurrent,nextInterestRatePercentageLocCurrent,originalLoanTermLocCurrent,vehicleValueAmountLocCurrent,obligorEmploymentVerificationCodeMLocCurrent,reportingPeriodBeginningLoanBalanceAmountLocCurrent,originatorNameLocCurrent,assetSubjectDemandIndicatorLocCurrent,interestCalculationTypeCodeMLocCurrent,paymentTypeCodeMLocCurrent,vehicleModelNameLocCurrent,vehicleModelYearLocCurrent,coObligorIndicatorLocCurrent,loanMaturityDateRLocCurrent,reportingPeriodModificationIndicatorLocCurrent,originalFirstPaymentDateRLocCurrent,scheduledInterestAmountLocCurrent,zeroBalanceEffectiveDateRLocCurrent,reportingPeriodScheduledPaymentAmountLocCurrent,originalInterestRatePercentageLocCurrent,reportingPeriodInterestRatePercentageLocCurrent,assetAddedIndicatorLocCurrent,vehicleValueSourceCodeMLocCurrent,reportingPeriodActualEndBalanceAmountLocCurrent,originalInterestOnlyTermNumberLocCurrent,servicingAdvanceMethodCodeMLocCurrent,vehicleTypeCodeMLocCurrent,remainingTermToMaturityNumberMinCurrent,remainingTermToMaturityNumberMaxCurrent,reportingPeriodEndingDateRMinCurrent,reportingPeriodEndingDateRMaxCurrent,reportingPeriodBeginningDateRMinCurrent,reportingPeriodBeginningDateRMaxCurrent,reportingPeriodBeginningLoanBalanceAmountMinCurrent,reportingPeriodBeginningLoanBalanceAmountMaxCurrent,obligorCreditScoreMinCurrent,obligorCreditScoreMaxCurrent,nextReportingPeriodPaymentAmountDueMinCurrent,nextReportingPeriodPaymentAmountDueMaxCurrent,gracePeriodNumberMaxCurrent,recoveredAmountMaxCurrent,chargedoffPrincipalAmountMaxCurrent,currentDelinquencyStatusMaxCurrent,interestPaidThroughDateRMaxCurrent,mostRecentServicingTransferReceivedDateRMaxCurrent,paymentExtendedNumberMaxCurrent,repossessedProceedsAmountMaxCurrent,accountStatusValsCurrent,assetSubjectDemandIndicatorValsCurrent,repurchaserNameValsCurrent,subventedMValsCurrent,modificationTypeCodeMValsCurrent,zeroBalanceCodeMValsCurrent,repossessedIndicatorValsCurrent,repurchaseReplacementReasonCodeMValsCurrent,assetSubjectDemandStatusCodeMValsCurrent,actualInterestCollectedAmountSumCurrent,gracePeriodNumberSumCurrent,repurchaseAmountSumCurrent,recoveredAmountSumCurrent,actualPrincipalCollectedAmountSumCurrent,repossessedProceedsAmountSumCurrent,actualOtherCollectedAmountSumCurrent,chargedoffPrincipalAmountSumCurrent,currentDelinquencyStatusSumCurrent,otherAssessedUncollectedServicerFeeAmountSumCurrent,totalActualAmountPaidSumCurrent,servicingFeePercentageSumCurrent,servicerAdvancedAmountSumCurrent,paymentExtendedNumberSumCurrent,otherServicerFeeRetainedByServicerSumCurrent,servicingFlatFeeAmountSumCurrent,eventOccurred,reportingPeriodEndingDateREvent,reportingPeriodBeginningDateREvent,accountStatusEvent,chargedoffPrincipalAmountEvent,zeroBalanceCodeMEvent,priorHistory,vehicleManufacturerNameLocPrior,primaryLoanServicerNameLocPrior,originalInterestRateTypeCodeMLocPrior,vehicleNewUsedCodeMLocPrior,originalLoanAmountLocPrior,obligorIncomeVerificationLevelCodeMLocPrior,underwritingIndicatorLocPrior,otherPrincipalAdjustmentAmountLocPrior,originationDateRLocPrior,obligorCreditScoreTypeLocPrior,obligorGeographicLocationLocPrior,paymentToIncomePercentageLocPrior,nextReportingPeriodPaymentAmountDueLocPrior,nextInterestRatePercentageLocPrior,originalLoanTermLocPrior,vehicleValueAmountLocPrior,obligorEmploymentVerificationCodeMLocPrior,reportingPeriodBeginningLoanBalanceAmountLocPrior,originatorNameLocPrior,assetSubjectDemandIndicatorLocPrior,interestCalculationTypeCodeMLocPrior,paymentTypeCodeMLocPrior,vehicleModelNameLocPrior,vehicleModelYearLocPrior,coOblig

In [31]:
master['securitization'] = term

### Export

In [32]:
e_folder = 'data/static/'
e_file = '{} static.csv'.format(term)
e_path = e_folder + e_file
e_path

'data/static/CarMax Auto Owner Trust 2017-1 Data Tape static.csv'

In [33]:
master.to_csv(e_path, index = False)

In [34]:
print('continue...')

continue...


### End